### 다중 분류에서 이진 분류로(구현)

In [30]:
import sys
sys.path.append('..')
from common.np import *  # import numpy as np
from common.layers import Embedding, SigmoidWithLoss
import collections


# 원래 CBOW의 거대한 행렬의 곱을 간소화한 모델
class EmbeddingDot:
    def __init__(self, W):
        self.embed = Embedding(W)
        self.params = self.embed.params
        self.grads = self.embed.grads
        self.cache = None

    def forward(self, h, idx):
        target_W = self.embed.forward(idx)
        out = np.sum(target_W * h, axis=1) #target_W * h 내적 구해져있음
        self.cache = (h, target_W)
        return out

    def backward(self, dout):
        h, target_W = self.cache
        dout = dout.reshape(dout.shape[0], 1)
        dtarget_W = dout * h
        self.embed.backward(dtarget_W)
        dh = dout * target_W
        return dh

In [35]:
W = np.arange(21).reshape(7,3)
h = np.arange(9).reshape(3,3)
idx = np.array([0,3,1])
embed = EmbeddingDot(W)
target_W = embed.forward(h, idx) #Wout에서 필요한 것만 꺼낸 것 
target_W

array([  5, 122,  86])

### 네거티브 샘플링의 샘플링 기법

In [2]:
import numpy as np

np.random.choice(10)

9

In [59]:
np.random.choice(10)

2

In [74]:
words = ['you','say','goodbye','I','hello','.']
np.random.choice(words)

'goodbye'

In [80]:
np.random.choice(words, size=5) # size만큼의 랜덤한 output이 나옴

array(['goodbye', '.', 'you', '.', '.'], dtype='<U7')

In [88]:
np.random.choice(words, size=5, replace=False) #replace=False 중복제거

array(['say', 'goodbye', 'I', 'hello', 'you'], dtype='<U7')

In [129]:
p = [0.5, 0.1, 0.05, 0.2, 0.05, 0.1] # you say goodbye I hello .
np.random.choice(words, p=p) #you 의 확률이 가장 높기때문에 you가 제일 많이 나옴

'you'

In [4]:
p = [0.7, 0.29, 0.01]
new_p = np.power(p, 0.75) # 확률이 너무 낮은 애들은 아예 뽑히지 않는 것을 방지하기 위해 0.75 power 보정치를 줌. 확률이 높은 애들은 0.75 power 했을 때 값이 낮아지지만 낮은 애들은 오히려 올라감. 0.75는 fixed된 값이 아니고 이 수가 낮아질수록 보정치가 높아짐
new_p /= np.sum(new_p)
print(new_p)

[0.65464647 0.32347952 0.02187401]


In [137]:
import collections

corpus = np.array([1,2,3,4,5,2,3,4,1,1,4,2,2,2])
counts = collections.Counter()
print(len(counts))
for word_id in corpus:
    counts[word_id] += 1
    
print(len(counts))
print(counts)

0
5
Counter({2: 5, 1: 3, 4: 3, 3: 2, 5: 1})


In [138]:
GPU = False

class UnigramSampler:
    def __init__(self, corpus, power, sample_size):
        self.sample_size = sample_size
        self.vocab_size = None
        self.word_p = None

        counts = collections.Counter()
        for word_id in corpus:
            counts[word_id] += 1

        vocab_size = len(counts)
        self.vocab_size = vocab_size

        self.word_p = np.zeros(vocab_size)
        for i in range(vocab_size):
            self.word_p[i] = counts[i]

        self.word_p = np.power(self.word_p, power)
        self.word_p /= np.sum(self.word_p)

    def get_negative_sample(self, target):
        batch_size = target.shape[0]

        if not GPU:
            negative_sample = np.zeros((batch_size, self.sample_size), dtype=np.int32)

            for i in range(batch_size):
                p = self.word_p.copy()
                target_idx = target[i]
                p[target_idx] = 0
                p /= p.sum() # 전체에서 자기자신이 차지하는 비울 (빈도 수)구하고 아래 random.choice의 확률로 넣어줌
                negative_sample[i, :] = np.random.choice(self.vocab_size, size=self.sample_size, replace=False, p=p) #랜덤하게 선택하기는 하나 확률 p=p가 가미되어있기 때문에 확률이 높은 애들은 선택확률이 높음
        else:
            # GPU(cupy）로 계산할 때는 속도를 우선한다.
            # 부정적 예에 타깃이 포함될 수 있다.
            negative_sample = np.random.choice(self.vocab_size, size=(batch_size, self.sample_size),
                                               replace=True, p=self.word_p)

        return negative_sample

In [149]:
corpus = np.array([0, 1, 2, 3, 4, 1, 2, 3])
power = 0.75
sample_size = 2

sampler = UnigramSampler(corpus, power, sample_size)
target = np.array([1, 3, 0])
negative_sample = sampler.get_negative_sample(target)
print(negative_sample)

[[4 2]
 [0 2]
 [4 2]]


In [28]:
a = np.array([1,2,3])
b = np.array([4,5,6])
c = np.c_[a,b]
print(c)

[[1 4]
 [2 5]
 [3 6]]


### 네거티브 샘플링 구현

In [150]:
class NegativeSamplingLoss:
    def __init__(self, W, corpus, power=0.75, sample_size=5): # 정답 하나에 부정샘플링 5개
        self.sample_size = sample_size
        self.sampler = UnigramSampler(corpus, power, sample_size)
        self.loss_layers = [SigmoidWithLoss() for _ in range(sample_size + 1)]
        self.embed_dot_layers = [EmbeddingDot(W) for _ in range(sample_size + 1)]

        self.params, self.grads = [], []
        for layer in self.embed_dot_layers:
            self.params += layer.params
            self.grads += layer.grads

    def forward(self, h, target):
        batch_size = target.shape[0]
        negative_sample = self.sampler.get_negative_sample(target)

        # 긍정적 예 순전파
        score = self.embed_dot_layers[0].forward(h, target)
        correct_label = np.ones(batch_size, dtype=np.int32)
        loss = self.loss_layers[0].forward(score, correct_label)

        # 부정적 예 순전파
        negative_label = np.zeros(batch_size, dtype=np.int32)
        for i in range(self.sample_size):
            negative_target = negative_sample[:, i]
            score = self.embed_dot_layers[1 + i].forward(h, negative_target)
            loss += self.loss_layers[1 + i].forward(score, negative_label)

        return loss

    def backward(self, dout=1):
        dh = 0
        for l0, l1 in zip(self.loss_layers, self.embed_dot_layers):
            dscore = l0.backward(dout)
            dh += l1.backward(dscore)

        return dh
